In [2]:
import gym
from gym import spaces
import numpy as np
from nasbench import api

In [ ]:
dataset = api.NASBench("/scratch2/sem22hs2/nasbench_full.tfrecord")

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [5]:
def objective_function(adjacency_mat,labeling, budget=108):
    labeling = ['input'] + list(labeling) + ['output']
    model_spec = api.ModelSpec(adjacency_mat, labeling)
    try:
        data = dataset.query(model_spec, epochs=budget)
    except api.OutOfDomainError:
        # self.record_invalid(adjacency_mat, labeling, 1, 1, 0)
        return 0, 0

    # self.record_valid(adjacency_mat, labeling, data, model_spec)
    return data["validation_accuracy"], data["training_time"]

In [6]:
# Todo: sample architecture
# Todo: Check architecture validity. Neg reward if not.

class NasBench101(gym.Env):
    metadata = {"render_modes": [], "render_fps": 1}
    def __init__(self, v=7, e=9, ops=['conv1x1-bn-relu', 'conv3x3-bn-relu', 'maxpool3x3'], step_max=1000, 
                 render_mode: Optional[str] = None):
        assert render_mode is None # or render_mode in self.metadata["render_modes"]
        # Environment definition
        self.max_edges = e
        self.vertices = v
        self.ops = ops

        # Current state
        self.adjacency_mat = np.zeros([v,v])
        self.labeling = (v-2)*[ops[0]] # Initialize op for all layers that are not input or output layer
        
        # Helper
        self.idx_upper = np.triu_indices(v) # Indices of upper triangular matrix

        self.num_step = 0
        self.step_max =  step_max
        num_indecies_triu = len(self.idx_upper[0])
        self.observation_space = spaces.Dict(
            {
                "adjacency_mat": spaces.MultiBinary(num_indecies_triu),
                "labels": spaces.MultiDiscrete(np.array((v-2)*[len(ops)])),
            }
        )

        
    def step(self, action):
        e=self.max_edges
        v=self.vertices
        n = (v*(v+1)/2) # Number of indices in upper triag. part of matrix
        if action < n:
            # Todo: Check this changes matrix at right place
            iu = self.idx_upper
            self.adjacency_mat[iu[0][action],iu[1][action]] = not self.adjacency_mat[iu[0][action],iu[1][action]]
        else:
            o=len(self.ops)
            action = action - n
            [label_row, op] = np.unravel_index(action,[v, o])
            self.labeling[label_row] = op

        y, c = objective_function(self.adjacency_mat, self.labeling)
        reward = y
        if self.step == self.step_max:
            done = 1
        else: done = 0

        observation = None
        info = None
        return observation, reward, done, info

    def reset(self):
        self.adjacency_mat[self.idx_upper] = np.random.randint(0,2,len(self.idx_upper))
        self.labeling = np.random.randint(0,3,len(self.labeling))

In [52]:
# Create the API instance for the topology search space in NATS
from nats_bench import create
api = create("/scratch2/sem22hs2/NATS-tss-v1_0-3ffb9-simple", 'tss', fast_mode=True, verbose=False)

In [ ]:
Show the architecture topology string of the 12-th architecture
# For the topology search space, the string is interpreted as
# arch = '|{}~0|+|{}~0|{}~1|+|{}~0|{}~1|{}~2|'.format(
#         edge_node_0_to_node_1,
#         edge_node_0_to_node_2,
#         edge_node_1_to_node_2,
#         edge_node_0_to_node_3,
#         edge_node_1_to_node_3,
#         edge_node_2_to_node_3,
#         )

In [76]:
architecture_str = api.arch(12)
print(architecture_str)

|none~0|+|none~0|none~1|+|none~0|nor_conv_3x3~1|avg_pool_3x3~2|


In [78]:
info = api.get_more_info(architecture_str, 'cifar10')
print(info["test-accuracy"]/100)


0.1


In [84]:
class NasBench201Tensor(gym.Env):
    metadata = {"render_modes": [], "render_fps": 1}
    def __init__(self, step_max=1000, render_mode: Optional[str] = None):
        assert render_mode is None # or render_mode in self.metadata["render_modes"]
        
        # Environment definition
        self.vertices = 4
        self.ops = ['nor_conv_1x1', 'nor_conv_3x3', 'avg_pool_3x3', 'skip_connect']

        # Current state
        v = self.vertices
        no_ops = len(self.ops)
        self.adjacency_tensor = np.zeros([v,v,no_ops])
        
        # Helper
        self.triu_y, self.triu_x = np.triu_indices(v, 1) # Indices of upper triangular matrix
        self.no_triu = len(self.triu_y[0]) # Number of upper triangular elements in matrix
        num_triu = len(self.self.no_triu)
        self._label_to_op = {
            0: 'nor_conv_1x1',
            1: 'nor_conv_3x3',
            2: 'avg_pool_3x3',
            3: 'skip_connect',
        }
        self.observation_space = spaces.MultiBinary(no_ops * num_triu)
        self.action_space = spaces.Discrete(no_ops * num_triu)
        
        # Len intervals
        self.num_step = 0
        self.step_max =  step_max
        
        if self.render_mode == "human":
            pass
        
        
    def _tensor2obs(self, tensor):
        obs=[]
        for i in range(adjacency_tensor.shape[2]):
            obs.extend(adjacency_tensor[self.triu_x,self.triu_y,i])
        return obs
    
    def _action2tensor(self, action):
        no_triu = self.no_triu
        tensor_z = int(action/no_triu)
        triu_idx = action%no_triu
        tensor_x = self.triu_x[triu_idx]
        tensor_y = self.triu_y[triu_idx]
        
        element =  self.observation_space[action]
        new_element = not element
        if new_element is True:
            # Enforce whole row to be zero
            self.adjacency_tensor[tensor_y, tensor_x, :] = np.zeros(4, dtype=int)
        self.adjacency_tensor[tensor_y, tensor_x, tensor_z] = new_element
        return
    
    def ten2str(tensor):
        adj_str = "|"
        for j in range(tensor.shape[1]): #to
            for k in range(tensor.shape[0]): #from
                if j <= k:
                    continue
                if k==0 and j != 1:
                    adj_str += "+|"
                    
                for i in range(tensor.shape[2]):#op
                    if not tensor[:,k,j].any():
                        adj_str += "none~"+str(k)+"|"
                        break
                    if tensor[i,k,j] == 1:
                        adj_str +=_label_to_op[i] +"~"+str(k)+ "|"
        return adj_str
    
    def nb201_lookup(tensor):
        arch=ten2str(tensor)
        acc=api.get_more_info(architecture_str, 'cifar10')
        return acc
                           
                       
    def step(self, action):
        # Determine new adjacency matrix and observation space
        _action2tensor(action)
        self.observation = self._tensor2obs(self.adjacency_tensor)
        # Calculate reward
        acc = nb201_lookup(self.adjacency_tensor)
        reward = acc
        if self.step == self.step_max:
            done = 1
        else: done = 0

        observation = self.observation
        info = None
        return observation, reward, done, info

    def reset(self):
        

IndentationError: expected an indented block (3114960485.py, line 96)

In [ ]:
import ray
from ray.tune.integration.wandb import WandbLoggerCallback

In [82]:
SELECT_ENV = NasBench201Tensor
N_ITER = 20


ray.init()
ray.tune.run(
    "PPO",
    stop={"training_iteration": 15},
    config={
        "env": SELECT_ENV,
        "record_env": True,
        "framework": "torch",
        "num_cpus_per_worker": 2,
        "num_gpus": 2,
        "num_workers": 4,
    },
    #local_dir="logs",
    #callbacks=[WandbLoggerCallback(api_key="c36c598399c6c7f2f0b446aac164da6c7956a263", project="NasBenchV1")],
)


trainer.evaluate()

NameError: name 'NasBench201Tensor' is not defined

In [9]:
ray.shutdown()

In [12]:
b = NasBench(v=3)

In [14]:
b.step(1)

(None, 0, 0, None)

In [16]:
b=[1,2,3]

In [16]:
import numpy as np
y,x =np.triu_indices(4,1)
mat=np.arange(16).reshape(4, 4)

In [20]:
mat[y[0],x[0]]=90

In [39]:
from gym import spaces
b=spaces.MultiBinary(3 * 3)

'henpl'

In [37]:
import numpy as np
p=np.array([[[0,1,0],[0,0,0],[0,0,0]], [[0,0,0],[0,0,1],[0,0,0]], [[0,0,0],[0,0,0],[0,0,0]]])
_label_to_op = {
            0: 'nor_conv_1x1',
            1: 'nor_conv_3x3',
            2: 'avg_pool_3x3',
            3: 'skip_connect',
        }

In [51]:
nb201_lookup(p)

0 2
[0 0 0]


'|nor_conv_1x1~0|+|none~0|nor_conv_3x3~1|'

In [49]:
print(p[:,1,2])
print(p)

[0 1 0]
[[[0 1 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 1]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]]]
